Model Formulation

- Capacity expansion problem with PSPS period and normal operations (perhaps start with just 2-4 rep days, and 1 PSPS day)
- Decision variables (solar + battery investment?) --> where are the costs for this? See code on github
- What should the costs of outage be?
- With reduced lines problem will be, potentially, infeasible. How will you add load shedding into the model? Can you find some papers on this?

In [31]:
using JuMP, HiGHS
using DataFrames, CSV

In [41]:
# Global path to test system datasets
PATH = joinpath(pwd(), "..", "data", "test_system") 

# Load each of the San Diego test system files 
gens = CSV.read(joinpath(PATH, "gens.csv"), DataFrame);
lines = CSV.read(joinpath(PATH, "lines.csv"), DataFrame);
loads = CSV.read(joinpath(PATH, "loads_24h_sp.csv"), DataFrame);

In [39]:
#=
Function to solve DC OPF problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function dcopf(gen, branch, gencost, bus)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets
      # Set of all generators
    G = gen.id
      # Set of all nodes
    N = bus.bus_i
      # sets J_i and G_i will be described using dataframe indexing below

    
    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = gen.mbase[1] # base MVA is 100 MVA for this system
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0
    # Note: by convention this is a generator bus. Hence, we will select bus 1
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gencost[g,:x1] * GEN[g] 
                        for g in G)
    )
    
    # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gen[gen.bus .== i,:id]) 
                - bus[bus.bus_i .== i,:pd][1] ==
        sum(FLOW[i,j] for j in branch[branch.fbus .== i,:tbus])
    )
    
    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gen[g,:pmax])

    # Flow constraints on each branch 
    # In DCOPF, line flow is a function of voltage angles
    @constraint(DCOPF, cLineFlows[l in 1:nrow(branch)],
            FLOW[branch[l,:fbus],branch[l,:tbus]] ==
            baseMVA*branch[l,:sus]*(THETA[branch[l,:fbus]] - THETA[branch[l,:tbus]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(branch)], 
           FLOW[branch[l,:fbus],branch[l,:tbus]] <= 
                branch[l,:ratea]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = branch.fbus,
        tbus = branch.tbus,
        flow = baseMVA .* branch.sus .* (angles[branch.fbus] .- angles[branch.tbus])
    )
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = bus.bus_i,
        value = dual.(cBalance).data)
    
    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

Row,Column1,OBJECTID,br_r,rate_a,Comments,br_b,Lon2,Lon1,br_x,kV,Lat1,f_bus,t_bus,CATS_ID,Lat2,transformer,Shape_Length,Length_km
,Int64,Int64,Float64,Float64,String?,Float64,Float64,Float64,Float64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Float64,Float64
1,5059,5059,0.000928922,0.64,,3.73247e-5,-117.643,-117.643,0.00245559,66,33.556,7986,5746,5059,33.5581,0,0.00212856,0.235943
2,6809,6809,0.00121225,0.64,,5.09106e-5,-117.643,-117.646,0.00307058,66,33.5576,5753,5746,6809,33.5581,0,0.00327911,0.307907
3,6810,6810,0.0033273,0.64,,0.000155506,-117.646,-117.655,0.00760939,66,33.5602,5752,5753,6810,33.5576,0,0.0089424,0.845125
4,8401,8401,0.000435858,10.56,,0.0111545,-117.59,-117.555,0.00482013,230,33.37,8309,7006,8401,33.4556,0,0.108486,11.4358
5,8403,8403,0.000804307,11.58,,0.022423,-117.643,-117.555,0.0108879,230,33.3699,8309,7985,8403,33.556,0,0.230705,24.4195
6,8405,8405,0.00179325,2.5,,0.0021695,-117.665,-117.625,0.0130196,115,33.4986,6777,731,8405,33.5128,0,0.0434571,4.13961
7,8407,8407,0.000496033,14.62,,0.0160804,-116.985,-116.91,0.00506398,230,32.5747,6778,8757,8407,32.6796,0,0.135582,14.1963
8,8408,8408,2.06049e-5,7.66,,0.000296281,-116.913,-116.91,0.000151524,230,32.5747,6778,8003,8408,32.574,0,0.00348426,0.330979
9,8409,8409,1.55646e-6,8.34,,7.20685e-5,-116.91,-116.91,1.86201e-5,230,32.5743,6779,6778,8409,32.5747,0,0.000420406,0.0448335


In [45]:
loads

Row,bus,2018-04-03 00:00:00-05:00,2018-04-03 01:00:00-05:00,2018-04-03 02:00:00-05:00,2018-04-03 03:00:00-05:00,2018-04-03 04:00:00-05:00,2018-04-03 05:00:00-05:00,2018-04-03 06:00:00-05:00,2018-04-03 07:00:00-05:00,2018-04-03 08:00:00-05:00,2018-04-03 09:00:00-05:00,2018-04-03 10:00:00-05:00,2018-04-03 11:00:00-05:00,2018-04-03 12:00:00-05:00,2018-04-03 13:00:00-05:00,2018-04-03 14:00:00-05:00,2018-04-03 15:00:00-05:00,2018-04-03 16:00:00-05:00,2018-04-03 17:00:00-05:00,2018-04-03 18:00:00-05:00,2018-04-03 19:00:00-05:00,2018-04-03 20:00:00-05:00,2018-04-03 21:00:00-05:00,2018-04-03 22:00:00-05:00,2018-04-03 23:00:00-05:00
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,708,4431.71,3725.12,3180.01,3017.44,2841.97,2809.66,2828.84,2939.41,3102.18,3512.68,3950.37,4507.97,4839.6,5117.95,5544.08,5963.02,6107.74,6140.16,6293.14,6428.73,6409.14,6002.74,5437.95,5028.28
2,731,4257.19,3573.12,3035.4,2889.4,2721.53,2678.68,2698.56,2775.82,2921.58,3292.63,3713.21,4296.33,4663.35,4966.89,5380.08,5807.31,5914.94,5903.32,6055.75,6190.86,6183.79,5746.14,5197.88,4834.94
3,732,12492.4,10023.6,8441.88,8019.1,7697.06,7567.32,7675.46,8083.25,8987.74,10313.8,11819.7,13345.2,14586.9,15505.3,16349.5,17115.8,17122.7,17138.4,17716.0,17988.4,17847.4,16793.6,15041.0,13976.6
4,733,8526.06,7156.05,6079.13,5786.73,5450.53,5364.71,5404.53,5559.25,5851.17,6594.3,7436.62,8604.46,9339.5,9947.41,10774.9,11630.6,11846.1,11822.8,12128.1,12398.7,12384.6,11508.0,10410.0,9683.15
5,734,7446.45,6238.68,5327.04,5070.16,4889.45,4860.9,4915.02,5135.4,5437.62,6204.23,7052.19,7713.14,8180.66,8504.48,9227.71,9943.46,10202.4,10341.0,10771.0,10983.6,11011.3,10166.8,9213.68,8548.21
6,1044,17471.5,14717.9,12988.5,11712.9,11166.4,11736.4,11880.8,12439.7,13876.2,16990.2,19061.3,21381.0,22785.8,24336.6,25526.6,25880.4,25122.1,23468.0,23287.2,22991.5,22897.5,22004.3,19920.4,19652.3
7,1045,4998.81,4274.73,3720.08,3551.98,3488.97,3481.61,3624.18,3865.11,4193.85,4891.48,5633.07,6169.24,6578.36,7093.69,7506.36,7530.22,7351.27,7417.2,7557.44,7239.3,6998.04,6485.15,6035.36,5718.23
8,1046,29271.9,24248.3,21266.2,19072.3,17795.3,18168.3,18769.5,19734.7,22466.2,27697.4,31120.1,37011.5,38988.2,41266.3,43028.5,42813.3,42200.8,39424.2,38672.6,39107.9,38905.1,37636.1,33943.3,33426.1
9,1047,1203.16,997.253,868.329,796.967,766.854,767.869,784.644,830.584,912.457,1107.7,1253.46,1426.92,1507.02,1591.14,1663.79,1699.76,1660.35,1583.43,1581.79,1583.95,1563.66,1508.44,1367.92,1340.6
